In [1]:
from datasets import load_dataset

stack_exchange_data = load_dataset("HuggingFaceH4/stack-exchange-preferences",split="train",streaming=True)

/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# stack_exchange_data.filter(lambda )

In [2]:
# next(iter(stack_exchange_data.shuffle()))


In [3]:
# stack_exchange_iterable_data = stack_exchange_data.take(1000_000).shuffle()
# for s in stack_exchange_iterable_data:
#     print(s)

In [4]:
from bs4 import BeautifulSoup

def get_text_html(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')

    paragraphs = soup.find_all('p')

    # Extract the text from each <p> tag
    paragraph_texts = [p.get_text() for p in paragraphs]

    # Print the extracted text
    answer_text = "".join([text for text in paragraph_texts])
    return answer_text


In [10]:
50_000/3600

13.88888888888889

In [5]:
import pandas as pd
from tqdm import tqdm

columns = ['date','qid','question','answer_text','selected_answer_text','stack_exchange']
df = pd.DataFrame(columns=columns)

# stack_exchanges= ['ai','bitcoin','anime',"datascience","astronomy"]
qids = []
dates = []
question_list = []
answer_text_list = []
selected_answer_text_list = []
stack_exchange_list = []
total = 50_000
progress_bar = tqdm(total=total)
curr_idx = 0
while True:
    if total == curr_idx: break
    sd = next(iter(stack_exchange_data.shuffle()))
    best_answer_pm_score_list = [s['pm_score'] for s in sd['answers']]
    if len(list(set(best_answer_pm_score_list))) < len(best_answer_pm_score_list): 
        progress_bar.update(1)
        continue
    if "meta" in sd['metadata'][0]:
        progress_bar.update(1)
        continue

    qids.append(sd['qid'])
    dates.append(sd['date'])
    link = sd['metadata'][1]
    stack_exchange_list.append(link[link.find("//"):link.find(".")][2:])
    question_list.append(get_text_html(sd['question']))
    answer_text = ""
    best_answer = ""
    best_answer_pm_score = 0
    for idx,ans in enumerate(sd['answers']):
        curr_answer = get_text_html(ans['text'])
        answer_text += curr_answer + "\n\n"
        if ans['pm_score'] > best_answer_pm_score:
            best_answer = curr_answer

    answer_text_list.append(answer_text)
    selected_answer_text_list.append(best_answer)
    progress_bar.update(1)
    curr_idx += 1

  0%|          | 19/50000 [00:18<8:53:36,  1.56it/s] 

KeyboardInterrupt: 

In [6]:

df['qid'] = qids
df['date'] = dates
df['question'] = question_list
df['answer_text'] = answer_text_list
df['selected_answer_text'] = selected_answer_text_list
df['stack_exchange'] = stack_exchange_list
df.head()

,date,qid,question,answer_text,selected_answer_text,stack_exchange
0,2014/11/12,3464,"This is a really nice feature, but it only dis...",You can use M-x blink-matching-open RET to dis...,That message is printed when the beginning of ...,emacs
1,2011/09/08,7344084,My site was working all fine on localhost but ...,Is your local machine Windows and your remote ...,Is your local machine Windows and your remote ...,Stackoverflow
2,2010/08/16,2584,I've got a fixed solution space defined by a m...,First let's make sure I understood your questi...,First let's make sure I understood your questi...,math
3,2017/05/18,306207,I am trying to amplify the voltage measured ac...,I think the small signal diagram is wrong. You...,If you want to find \$ R_{OUT}\$ we need to so...,electronics
4,2016/01/04,34585362,I'm having one null-able bool (bool?) variable...,Use x.GetValueOrDefault() to assign default va...,Use x.GetValueOrDefault() to assign default va...,Stackoverflow


In [8]:
df['selected_answer_text'].values

array(['That message is printed when the beginning of the sexp is not visible in the window; otherwise, the opening paren is temporarily highlighted. This behavior is implemented in simple.el and is not associated with any minor mode. It is implemented using a hook that runs when a character is inserted. The actual blinking/message comes from command blink-matching-open, which you can call yourself. For example you could start an idle-timer that will call this command whenever you pause for a short time on a closing paren. This is the way that the built-in show-paren-mode works.',
       'Is your local machine Windows and your remote server a *nix box?  In that case, check your capitalization.  CHTML.php suggests that you have case-mismatches, try using CHtml in your code instead.',
       "First let's make sure I understood your question correctly:Is this correct?If this is the case, I reccommend you use a variation of rejection sampling. The recipe is quite straightforward, but you m

In [26]:
df['stack_exchange'].value_counts()

stack_exchange
Stackoverflow    489681
3dprinting         1283
Name: count, dtype: int64

In [43]:
import csv
df.to_csv("StackExchangeData.csv",index=False,quoting=csv.QUOTE_NONE, escapechar='\\')

: 

In [37]:
print(len(df['answer_text'].values[0].split("\n\n\n\n")))

3


In [42]:
df['answer_text'].values[0].split("\n\n\n\n")[-1]

'\n'

In [2]:
import pandas as pd
import csv
reader = pd.read_csv("StackExchangeData.csv", iterator=True,quoting=csv.QUOTE_NONE, escapechar='\\')
df = reader.get_chunk(10**6)

In [8]:
len(df)

stack_exchange
Stackoverflow    452860
3dprinting         1283
Name: count, dtype: int64

In [16]:
print(df['answer_text'][1])

As with any manufacturing process, you'll need to learn to "use the right tool for the job". It depends on the requirements of the part. To answer your question, I would suggest using a larger layer height for the sheer fact of reducing print time on larger objects.
However, it depends on the part and how small the details are on the part. If your part has sharp edges that are required for the proper functionality of the part, then you'll want to use a smaller layer height. Or if your part fits into another part, you'll probably want to use a smaller layer height.
Another variable might be whether or not post-processing is necessary. Is this part going to be purposefully printed larger/rougher with the idea to use a Dremel later to smooth everything out? If yes, then use a larger layer height.




I believe the Slic3r Manual summarizes this quite nicely:

A thicker layer height will provide more flow, and consequently more heat, making the extrusion adhere to the bed more. It also give